In [1]:
%load_ext autoreload

In [2]:
from certified_iris_generator import CertifiedIrisRegionGenerator
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from sandbox import rrtiris

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint, RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram, RigidTransform, RollPitchYaw
import meshcat
from pydrake.all import GenerateSeedingPolytope, GenerateRandomSeedingPolytope
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.multibody.rational_forward_kinematics import FindEpsilonLower, FindEpsilonLowerVector, FindEpsilonUpperVector
from pydrake.solvers import mathematicalprogram as mp
from pydrake.all import MosekSolver, MosekSolverDetails, FindEpsTilCollisionOrRedundantForAllIneqs
from time import time

# Build Plant


In [4]:
q0 = [0.0, 0.0, 0.0]
q_low  = [-1.7, -2., -1.7]
q_high = [ 1.7,  2.,  1.7]


In [5]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/oneDOF_iiwa7_with_box_collision.sdf")
twoDOF_iiwa_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/twoDOF_iiwa7_with_box_collision.sdf")

In [6]:
box_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/box_small.urdf")

models = []
models.append(parser.AddModelFromFile(box_asset))
models.append(parser.AddModelFromFile(twoDOF_iiwa_asset))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset))



locs = [[0.,0.,0.],[0.,.5,0.],[0.,-.5,0.]]
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", models[0]), RigidTransform(locs[0]))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_twoDOF_link_0", models[1]), RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[1]))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[2]))


plant.Finalize()

idx = 0
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
            
#q_low = plant.GetPositionLowerLimits().tolist()
#q_high = plant.GetPositionUpperLimits().tolist()

Ratfk = RationalForwardKinematics(plant)


In [7]:
do_viz = True
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram
visualizer.visualize_collision_constraint(N = 30)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7018/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7019/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6018...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7018/static/
Connected to meshcat-server.


In [8]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    visualizer.showres(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

visualizer.jupyter_cell()

FloatSlider(value=0.0, description='q0', max=1.7, min=-1.7)

FloatSlider(value=0.0, description='q1', max=2.0, min=-2.0)

FloatSlider(value=0.0, description='q2', max=1.7, min=-1.7)

In [9]:
#filter fused joints self collisions 
digaram = visualizer.diagram
context = visualizer.diagram_context
plant_context = visualizer.plant_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
get_name_of_gid = lambda gid : inspector.GetName(gid)
gids.sort(key=get_name_of_gid)
iiwa_oneDOF_gids = [gid for gid in gids if "iiwa7_oneDOF::" in get_name_of_gid(gid)]
iiwa_twoDOF_gids = [gid for gid in gids if "iiwa7_twoDOF::" in get_name_of_gid(gid)]

oneDOF_fused_col_geom = iiwa_oneDOF_gids[2:]
iiwa_oneDOF_fused_set = GeometrySet(oneDOF_fused_col_geom)
twoDOF_fused_col_geom = iiwa_twoDOF_gids[4:]
iiwa_twoDOF_fused_set = GeometrySet(twoDOF_fused_col_geom)
# print([get_name_of_gid(gid) for gid in oneDOF_fused_col_geom])
# print([get_name_of_gid(gid) for gid in twoDOF_fused_col_geom])
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_oneDOF_fused_set))
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_twoDOF_fused_set))
pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

150 -> 150
137 -> 150


# Setup IRIS Options and Generate Regions

In [10]:
# seed_points_q = np.array([[0.0, 0, 0], # zero config
#                         [0.0, -1.3, -1.3],  # start
#                         [0.9, -1.5, -0.8]     # goal
#                          ])
seed_points_q = np.array([[0.0, 0, 0], # zero config
                        [0.8, 1.3, -0.8],  # START: blue low green up
                        [0.1, 0.9, -1.2],     # GOAL: green low other up
                        [0.2, 1.6, -0.6],
                        [0.5, -1.9, -0.9],
                          [-0.6,-1.3,-0.3],
                          [-0.9,-1.7,-1.3],
                          [1.30, 1.50, -0.1],
                         ])[:,(0,1,2)]   # passing


seed_points = np.array([Ratfk.ComputeTValue(seed_points_q[idx], np.zeros((3,)))\
                        for idx in range(seed_points_q.shape[0])])
if do_viz:
    visualizer.plot_seedpoints(seed_points)

start = seed_points[1,:]
goal = seed_points[2,:]

#compute limits in t-space
limits_t = []
for q in [q_low, q_high]:
    limits_t.append(Ratfk.ComputeTValue(np.array(q), np.zeros((3,)) ))
    
starting_poly = HPolyhedron.MakeBox(limits_t[0], limits_t[1])
context = diagram.CreateDefaultContext()
q_star = np.zeros(3)
def check_empty_polytope(C, d):
    prog = MathematicalProgram()
    t = prog.NewContinuousVariables(C.shape[1], "t")

    prog.AddLinearConstraint(C,-np.inf*np.ones(C.shape[0]), d, t)
    prog.AddBoundingBoxConstraint(limits_t[0], limits_t[1], t)
    result = MosekSolver().Solve(prog)
    return result.is_success(), result.get_solution_result()

In [11]:
# RUN THIS CELL IF YOU WANT TO DO SNOPT IRIS

# regions = []

# iris_options = IrisOptionsRationalSpace()
# iris_options.require_sample_point_is_contained = True
# iris_options.iteration_limit = 20
# iris_options.configuration_space_margin = 1e-5
# iris_options.max_faces_per_collision_pair = 60
# iris_options.termination_threshold = -1
# iris_options.q_star = np.zeros(3)
# iris_options.relative_termination_threshold = 0.05
# iris_options.enable_ibex = False
# #deprecated
# iris_options.certify_region_with_sos_during_generation = False
# iris_options.certify_region_with_sos_after_generation = False

# for i, s in enumerate(seed_points):
#     plant.SetPositions(plant.GetMyMutableContextFromRoot(context), s)
#     if False:
#         #starting_hpolyhedron = regions[i-1]
#         r = IrisInRationalConfigurationSpace (plant, plant.GetMyContextFromRoot(context),
#                                               iris_options, starting_hpolyhedron)
#     else:
#         r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), iris_options)
#     regions.append(r)
#     print(f'Completed region: {i+1}/{len(seed_points)}')
#     print(f"Sample point contained = {np.all(r.A()@s <= r.b())}")
#     print(f"Sample point contained componentwise = {r.A()@s <= r.b()}")
#     print()

In [12]:
# if do_viz:
#     visualizer.plot_regions(regions, ellipses=None, region_suffix='original')

## Create default region

In [13]:
regions_default = []
max_vects = []
min_vects = []
vector_bisection_options_list = []
bilinear_alteration_options_list = []
interleaved_options_list = []

#parameters for creating a default region. Currently set to create a box
# num_perm_dim = 0
# num_rot = 1

num_unit_box_copies = 2
initial_box_scale = 0.25
gaussian_variance = 0.1

for i, s in enumerate(seed_points):
#     regions_default.append(GenerateSeedingPolytope(s, num_perm_dim, num_rot))
    regions_default.append(GenerateRandomSeedingPolytope(s,
                                     num_unit_box_copies,
                                     initial_box_scale,
                                     gaussian_variance))
#     eps_min_vect = FindEpsilonLowerVector(regions_default[i].A(), 
#                                       regions_default[i].b(), 
#                                       limits_t[0], limits_t[1], s)
    
#     min_vects.append(eps_min_vect)
    
#     eps_max_vect = FindEpsTilCollisionOrRedundantForAllIneqs(plant,
# #                                                              context,
#                                                              plant.GetMyMutableContextFromRoot(context), 
#                                 q_star, regions_default[i].A(), regions_default[i].b(), 
# #                                          eps_min_vect, 
#                                                              limits_t[0], limits_t[1], s)
#     max_vects.append(eps_max_vect)
    
#     vector_bisection_search_options = rational_forward_kinematics.VectorBisectionSearchOption()
#     vector_bisection_search_options.max_iters = 10
#     vector_bisection_search_options.max_feasible_iters = 5
#     vector_bisection_search_options.search_d = True
#     vector_bisection_search_options.epsilon_min = eps_min_vect
#     vector_bisection_search_options.epsilon_max = eps_max_vect
#     vector_bisection_search_options.num_threads = -1
#     vector_bisection_options_list.append(vector_bisection_search_options)
    
#     bilinear_alternation_options = rational_forward_kinematics.BilinearAlternationOption()
#     bilinear_alternation_options.max_iters = 20
#     bilinear_alternation_options.lagrangian_backoff_scale = 0
#     bilinear_alternation_options.polytope_backoff_scale = 0
#     bilinear_alternation_options.num_threads = -1
#     bilinear_alteration_options_list.append(bilinear_alternation_options)
    
#     interleaved_options = rational_forward_kinematics.InterleavedRegionSearchOptions()
#     interleaved_options.vector_bisection_search_options = vector_bisection_search_options
#     interleaved_options.bilinear_alternation_options = bilinear_alternation_options
#     #actually does round robin
#     interleaved_options.use_vector_bisection_search = True
#     interleaved_options.max_method_switch = 1
#     interleaved_options_list.append(interleaved_options)
    
    
num_round_robin_rounds = 1

if do_viz:
    visualizer.plot_regions(regions_default, ellipses=None, region_suffix='_default')
    

In [14]:
i = 7
s = seed_points[i,:]
C, d = regions_default[i].A(), regions_default[i].b()

eps_min_vect = FindEpsilonLowerVector(regions_default[i].A(), 
                                      regions_default[i].b(), 
                                      limits_t[0], limits_t[1], s)
eps_max_vect = FindEpsTilCollisionOrRedundantForAllIneqs(plant,plant.GetMyMutableContextFromRoot(context), 
                                q_star, regions_default[i].A(), regions_default[i].b()+eps_min_vect, 
                                                    limits_t[0], limits_t[1], s)


vector_bisection_search_options = rational_forward_kinematics.VectorBisectionSearchOption()
vector_bisection_search_options.max_iters = 10
vector_bisection_search_options.max_feasible_iters = 5
vector_bisection_search_options.search_d = True
vector_bisection_search_options.epsilon_min = eps_min_vect
vector_bisection_search_options.epsilon_max = eps_max_vect
vector_bisection_search_options.num_threads = -1
vector_bisection_options_list.append(vector_bisection_search_options)

bilinear_alternation_options = rational_forward_kinematics.BilinearAlternationOption()
bilinear_alternation_options.max_iters = 20
bilinear_alternation_options.lagrangian_backoff_scale = 0
bilinear_alternation_options.polytope_backoff_scale = 0
bilinear_alternation_options.num_threads = -1
bilinear_alteration_options_list.append(bilinear_alternation_options)

interleaved_options = rational_forward_kinematics.InterleavedRegionSearchOptions()
interleaved_options.vector_bisection_search_options = vector_bisection_search_options
interleaved_options.bilinear_alternation_options = bilinear_alternation_options
#actually does round robin
interleaved_options.use_vector_bisection_search = True
interleaved_options.max_method_switch = 1

# vector_bisection_search_options = vector_bisection_options_list[i]
# interleaved_options = interleaved_options_list[i]
r_moved_list = []
for j in range(len(d)):
    d_new = d.copy()
    d_new[j] += eps_max_vect[j]
    r_moved_list.append(HPolyhedron(C, d_new))
if do_viz:
    visualizer.plot_regions(r_moved_list, ellipses=None, region_suffix=f'_default_{i}_maxed')
if do_viz:
    visualizer.plot_regions([HPolyhedron(C, d+0.99*eps_max_vect)], ellipses=None, region_suffix=f'_all_{i}_maxed')

In [15]:
print(C.shape)
print(d.shape)
print(eps_max_vect.shape)
print(eps_min_vect.shape)

(12, 3)
(12,)
(12,)
(12,)


In [16]:
print(vector_bisection_search_options.epsilon_max)
print(vector_bisection_search_options.epsilon_min)
print(vector_bisection_search_options.epsilon_max-
     vector_bisection_search_options.epsilon_min)

[ 1.53466129e-11  5.95505867e-11  2.95490576e-11  4.19286827e-10
  1.63105085e-11 -6.64099331e-12  6.95199454e-12  5.92816907e-11
  3.45434098e-10  1.12046261e-10  3.43347573e-12  2.14762541e-10]
[-0.25000001 -0.24999999 -0.25       -0.24999999 -0.25000001 -0.25000001
 -0.25000001 -0.24999999 -0.24999999 -0.24999999 -0.25000001 -0.25000001]
[0.25000001 0.24999999 0.25       0.24999999 0.25000001 0.25000001
 0.25000001 0.24999999 0.24999999 0.24999999 0.25000001 0.25000001]


In [17]:
# #plot the new polytopes which each individual face moved
# if do_viz:
#     for i, r in enumerate(regions_default):
#         C, d = r.A(), r.b()
#         eps_max = vector_bisection_options_list[i].epsilon_max
#         r_moved_list = []
#         for j in range(len(d)):
#             d_new = d.copy()
#             d_new[j] += eps_max[j]
#             r_moved_list.append(HPolyhedron(C, d_new))
#         if do_viz:
#             visualizer.plot_regions(r_moved_list, ellipses=None, region_suffix=f'_default_{i}_maxed')
#         if do_viz:
#             visualizer.plot_regions([HPolyhedron(C, d+0.99*eps_max)], ellipses=None, region_suffix=f'_all_{i}_maxed')

## Run Cspace Free Region

In [18]:
cspace_free_region = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)
filtered_collision_pairs = set()
solver_options = mp.SolverOptions()
# solver_options.SetOption(mp.CommonSolverOption.kPrintToConsole, 1)

In [ ]:
t0 = time()
cspace_free_region_solution_interleaved = cspace_free_region.InterleavedCSpacePolytopeSearch(q_star,
                                                                     filtered_collision_pairs,
                                                                     C, d, num_round_robin_rounds, 
                                                                     interleaved_options,
                                                                     solver_options, s, 
                                                                     context = plant.GetMyMutableContextFromRoot(context))
t1 = time()


[2022-02-09 15:55:05.757] [console] [info] Starting bisections
[2022-02-09 15:55:17.865] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 15:55:20.720] [console] [info] bilinear alt on d succeeded
[2022-02-09 15:55:20.766] [console] [info] Ineq 1/12 in round 1
[2022-02-09 15:56:03.808] [console] [info] current gap is epsilon_gap=0.010000000000000009
[2022-02-09 15:56:03.809] [console] [info] current gap is epsilon_gap=0.010000000000000009
[2022-02-09 15:56:11.767] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 15:56:15.454] [console] [info] bilinear alt on d succeeded
[2022-02-09 15:56:15.521] [console] [info] epsilon=-0.24499908148427496 is feasible
[2022-02-09 15:57:02.792] [console] [info] reset eps_min=-0.2449990807925453, eps_max=-0.23499908079254528


In [29]:
print(f"Completed points in {t1-t0}s")

Completed points in 107.27950668334961s


In [31]:
r = HPolyhedron(np.vstack([starting_poly.A(),cspace_free_region_solution_interleaved.C]),
                np.hstack([starting_poly.b(), cspace_free_region_solution_interleaved.d])
                )
e = Hyperellipsoid(np.linalg.inv(cspace_free_region_solution_interleaved.P), 
                   cspace_free_region_solution_interleaved.q)
if do_viz:
    visualizer.plot_regions([r], 
                            ellipses=[e], region_suffix='_interleaved_single')

In [34]:
check_empty_polytope(r.A(), r.b())

(True, <SolutionResult.kSolutionFound: 0>)

In [23]:
# r = HPolyhedron(np.vstack([starting_poly.A(),cspace_free_region_solution_round_robin.C]),
#                 np.hstack([starting_poly.b(), cspace_free_region_solution_round_robin.d])
#                 )
# e = Hyperellipsoid(np.linalg.inv(cspace_free_region_solution_round_robin.P), 
#                    cspace_free_region_solution_round_robin.q)
# if do_viz:
#     visualizer.plot_regions([r], 
#                             ellipses=[e], region_suffix='_round_robin_single')

In [18]:
seed_point_list = [s for s in seed_points] 

C_mats = [r.A() for r in regions_default]
d_vects = [r.b() for r in regions_default]
t0 = time()
cspace_free_region_solution_interleaved_vect = cspace_free_region.InterleavedCSpacePolytopeSearchForSeedPoints(
                                                                    q_star,
                                                                     filtered_collision_pairs,
                                                                     C_mats, d_vects, num_round_robin_rounds, 
                                                                     interleaved_options_list,
                                                                     solver_options, seed_point_list)
# cspace_free_region_solution_round_robin_vect = cspace_free_region.CspacePolytopeRoundRobinBisectionSearchForSeedPoints(
#                                                                     q_star,
#                                                                      filtered_collision_pairs,
#                                                                      C_mats, d_vects, num_round_robin_rounds, 
#                                                                      vector_bisection_options_list,
#                                                                      solver_options, seed_point_list)
t1 = time()


[2022-02-09 10:45:59.308] [console] [info] Starting bisections
[2022-02-09 10:45:59.308] [console] [info] Starting bisections
[2022-02-09 10:45:59.308] [console] [info] Starting bisections
[2022-02-09 10:45:59.308] [console] [info] Starting bisections
[2022-02-09 10:45:59.309] [console] [info] Starting bisections
[2022-02-09 10:45:59.309] [console] [info] Starting bisections
[2022-02-09 10:45:59.309] [console] [info] Starting bisections
[2022-02-09 10:45:59.309] [console] [info] Starting bisections
[2022-02-09 10:46:35.194] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 10:46:35.422] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 10:46:35.422] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 10:46:35.423] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 10:46:35.461] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 10:46:35.488] [cons

[2022-02-09 10:47:05.280] [console] [info] epsilon=0.22074941265023962 is infeasible
[2022-02-09 10:47:05.281] [console] [info] current gap is epsilon_gap=0.3207494017405328
[2022-02-09 10:47:07.216] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_5_collision, iiwa7_oneDOF::iiwa_oneDOF_link_4_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_6_collision, iiwa7_oneDOF::iiwa_oneDOF_link_3_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_6_collision, iiwa7_oneDOF::iiwa_oneDOF_link_4_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_7_collision, iiwa7_oneDOF::iiwa_oneDOF_link_3_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_7_collision, iiwa7_oneDOF::iiwa_oneDOF_link_4_collision)

[2022-02-09 10:47:07.230] [console] [info] epsilon=0.3953594123772164 is infeasible
[2022-02-09 10:47:07.232] [console] [info] current gap is epsilon_gap=0.49535941417568696
[2022-02-09 10:47:07.511] [console] [info] Found Lagrangian multiplier and separating planes
[2022-

[2022-02-09 10:47:30.918] [console] [info] epsilon=0.5388501880304035 is infeasible
[2022-02-09 10:47:30.919] [console] [info] current gap is epsilon_gap=0.124163143053856
[2022-02-09 10:47:31.130] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 10:47:31.366] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box2_Collision@0000000004502418, iiwa7_twoDOF::iiwa_twoDOF_link_5_collision)
(box_scene::box2_Collision@0000000004502418, iiwa7_twoDOF::iiwa_twoDOF_link_6_collision)
(box_scene::box2_Collision@0000000004502418, iiwa7_twoDOF::iiwa_twoDOF_link_7_collision)

[2022-02-09 10:47:31.379] [console] [info] epsilon=0.7859110714444248 is infeasible
[2022-02-09 10:47:31.382] [console] [info] current gap is epsilon_gap=0.37087534697256364
[2022-02-09 10:47:35.101] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 10:47:35.102] [console] [warning] Cannot find Lagrangian multiplier and separatin

[2022-02-09 10:47:51.715] [console] [info] current gap is epsilon_gap=0.43365120611537245
[2022-02-09 10:47:54.475] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box4_Collision@000000000450dbb8, iiwa7_oneDOF::iiwa_oneDOF_link_4_collision)
(box_scene::box4_Collision@000000000450dbb8, iiwa7_oneDOF::iiwa_oneDOF_link_5_collision)
(box_scene::box4_Collision@000000000450dbb8, iiwa7_oneDOF::iiwa_oneDOF_link_6_collision)
(box_scene::box4_Collision@000000000450dbb8, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)

[2022-02-09 10:47:54.478] [console] [info] epsilon=1.0558361623563055 is infeasible
[2022-02-09 10:47:54.479] [console] [info] current gap is epsilon_gap=0.31243745432707937
[2022-02-09 10:47:54.671] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box2_Collision@0000000004502418, iiwa7_twoDOF::iiwa_twoDOF_link_4_collision)
(box_scene::box2_Collision@0000000004502418, iiwa7_twoDOF::iiwa_twoDOF_link_

[2022-02-09 10:48:16.818] [console] [info] current gap is epsilon_gap=0.054206400764421514
[2022-02-09 10:48:16.877] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 10:48:17.156] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 10:48:21.672] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 10:48:21.684] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 10:48:21.692] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box2_Collision@0000000004502418, iiwa7_twoDOF::iiwa_twoDOF_link_5_collision)
(box_scene::box2_Collision@0000000004502418, iiwa7_twoDOF::iiwa_twoDOF_link_6_collision)
(box_scene::box2_Collision@0000000004502418, iiwa7_twoDOF::iiwa_twoDOF_link_7_collision)

[2022-02-09 10:48:21.693] [console] [info] epsilon=0.6838485013305681 is infeasible
[2022-02-09 10:48:21.693] [console] [info] current gap is epsilon_gap=0.080979

...

[2022-02-09 10:58:07.907] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_6_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)

[2022-02-09 10:58:07.911] [console] [info] epsilon=-0.09980286290293021 is infeasible
[2022-02-09 10:58:07.914] [console] [info] current gap is epsilon_gap=0.00019530892935994137
[2022-02-09 10:58:09.115] [console] [info] Iter: 3, polytope step cost -0.015202817799817551, solver time 9.318045854568481
[2022-02-09 10:58:11.961] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_5_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)

[2022-02-09 10:58:11.962] [console] [info] epsilon=-0.09960821467948441 is infeasible
[2022-02-09 10:58:11.968] [console] [info] current gap is epsilon_gap=0.0003906204497234611
[2022-02-09 10:58:14.957] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 10:58:14.959] [conso

[2022-02-09 10:58:34.770] [console] [info] max(power(det(P), 1/4))=0.3355118330772159, solver_time 0.01621699333190918
[2022-02-09 10:58:34.773] [console] [info] cost improvement inf
[2022-02-09 10:58:34.800] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_7_collision, iiwa7_oneDOF::iiwa_oneDOF_link_2_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_7_collision, iiwa7_oneDOF::iiwa_oneDOF_link_3_collision)

[2022-02-09 10:58:34.800] [console] [info] epsilon=-0.07498960112248954 is infeasible
[2022-02-09 10:58:34.801] [console] [info] current gap is epsilon_gap=0.02499653370749652
[2022-02-09 10:58:35.040] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_5_collision, iiwa7_oneDOF::iiwa_oneDOF_link_4_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_6_collision, iiwa7_oneDOF::iiwa_oneDOF_link_3_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_6_collision, iiwa7_oneDOF::iiwa_oneDOF

[2022-02-09 10:59:04.886] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 10:59:05.401] [console] [info] Iter: 7, polytope step cost -0.016681018313228945, solver time 4.669577121734619
[2022-02-09 10:59:07.348] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 10:59:07.349] [console] [info] Lagrangian step time 1.929 s
[2022-02-09 10:59:07.354] [console] [info] max(power(det(P), 1/4))=0.37181952840454546, solver_time 0.0034089088439941406
[2022-02-09 10:59:07.355] [console] [info] cost improvement 0.003981167176576927
[2022-02-09 10:59:07.700] [console] [info] Iter: 0, polytope step cost -0.006438426884081421, solver time 2.241502046585083
[2022-02-09 10:59:09.531] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 10:59:09.532] [console] [info] Lagrangian step time 1.814 s
[2022-02-09 10:59:09.537] [console] [info] max(power(det(P), 1/4))=0.2247671273989768, solver_time 0.0028908252716064453
[2022

[2022-02-09 10:59:38.591] [console] [info] cost improvement 0.0031579312993427755
[2022-02-09 10:59:39.046] [console] [info] Iter: 5, polytope step cost -0.005441797323048607, solver time 2.271266222000122
[2022-02-09 10:59:40.835] [console] [info] Iter: 11, polytope step cost -0.01100945048868392, solver time 1.9345688819885254
[2022-02-09 10:59:41.160] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 10:59:41.165] [console] [info] Lagrangian step time 2.094 s
[2022-02-09 10:59:41.172] [console] [info] max(power(det(P), 1/4))=0.23367838080558967, solver_time 0.004619121551513672
[2022-02-09 10:59:41.173] [console] [info] cost improvement 0.001600156850170592
[2022-02-09 10:59:42.669] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 10:59:42.670] [console] [info] Lagrangian step time 1.804 s
[2022-02-09 10:59:42.683] [console] [info] max(power(det(P), 1/4))=0.4055110288607325, solver_time 0.006557941436767578
[2022-02-09 10:59

[2022-02-09 11:00:17.915] [console] [info] max(power(det(P), 1/4))=0.24069830086002197, solver_time 0.0029420852661132812
[2022-02-09 11:00:17.915] [console] [info] cost improvement 0.0012694839057764695
[2022-02-09 11:00:18.114] [console] [info] Iter: 17, polytope step cost -0.011747482768028809, solver time 1.675779104232788
[2022-02-09 11:00:19.908] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 11:00:19.909] [console] [info] Lagrangian step time 1.778 s
[2022-02-09 11:00:19.919] [console] [info] max(power(det(P), 1/4))=0.40060921573284847, solver_time 0.003988981246948242
[2022-02-09 11:00:19.920] [console] [info] cost improvement 0.0020867153980728137
[2022-02-09 11:00:20.896] [console] [info] Iter: 11, polytope step cost -0.004256169403008982, solver time 0.3764500617980957
[2022-02-09 11:00:21.149] [console] [info] Iter: 17, polytope step cost -0.008688459854710902, solver time 0.35751795768737793
[2022-02-09 11:00:24.801] [console] [info] Found L

In [65]:
print(len(seed_points))

8


In [62]:
print(seed_points)

[[ 0.          0.          0.        ]
 [ 0.42279322  0.7602044  -0.42279322]
 [ 0.05004171  0.48305507 -0.68413681]
 [ 0.10033467  1.02963856 -0.30933625]
 [ 0.25534192 -1.39838259 -0.48305507]
 [-0.30933625 -0.7602044  -0.15113522]
 [-0.48305507 -1.13833271 -0.7602044 ]
 [ 0.7602044   0.93159646 -0.05004171]]


In [19]:
print(f"Completed points in {t1-t0}s")
regions_round_robin = []
ellipses_round_robin = []
for sol in cspace_free_region_solution_interleaved_vect:
    r = HPolyhedron(np.vstack([starting_poly.A(),sol.C]),
                np.hstack([starting_poly.b(), sol.d])
                )
    e = Hyperellipsoid(np.linalg.inv(sol.P), sol.q)

    regions_round_robin.append(r)
    ellipses_round_robin.append(e)
if do_viz:
    visualizer.plot_regions(regions_round_robin, 
                            ellipses=None, region_suffix='_interleaved2')

Completed points in 880.2137861251831s


In [72]:
visualizer.vis2["regions_interleaved"]

<Visualizer using: <meshcat.visualizer.ViewerWindow object at 0x7f71c76cf190> at path: <meshcat.path.Path object at 0x7f718dce99a0>>

# Don't run after here

In [36]:
cspace_free_region_solution_round_robin = cspace_free_region.CspacePolytopeRoundRobinBisectionSearch(q_star,
                                                                     filtered_collision_pairs,
                                                                     C, d, 1, 
                                                                     vector_bisection_search_options,
                                                                     solver_options, s)

[2022-02-09 11:46:31.589] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 11:46:34.425] [console] [info] bilinear alt on d succeeded
[2022-02-09 11:46:34.475] [console] [info] Ineq 1/12 in round 1
[2022-02-09 11:46:34.476] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-09 11:46:34.491] [console] [info] output of eps_redundant_max is -3.436515294552578e-09
[2022-02-09 11:46:34.492] [console] [warning] round robin bisection search failed: epsilon max is less than epsilon min.
 difference is = 
     -0.25
-0.238571
-0.225191
 -0.20519
    -0.25
    -0.25
-0.239698
    -0.25
    -0.25
    -0.25
    -0.25
    -0.25
[2022-02-09 11:46:34.493] [console] [info] Ineq 2/12 in round 1
[2022-02-09 11:46:34.511] [console] [info] output of eps_redundant_max is 6.756142090225126e-09
[2022-02-09 11:46:34.512] [console] [info] Ineq 3/12 in round 1
[2022-02-09 11:46:34.529] [console] [info] output of eps_redund

[2022-02-09 11:47:44.971] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 11:47:47.809] [console] [info] bilinear alt on d succeeded
[2022-02-09 11:47:47.841] [console] [info] epsilon=-0.1569430869108956 is feasible
[2022-02-09 11:47:47.842] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-09 11:47:47.851] [console] [info] output of eps_redundant_max is 0.0007903398544878426
[2022-02-09 11:47:47.852] [console] [info] reset eps_min=-0.15694308925800982, eps_max=-0.15615274940352197
[2022-02-09 11:47:47.853] [console] [info] current gap is epsilon_gap=0.0007903398544878426
[2022-02-09 11:47:53.058] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 11:47:55.860] [console] [info] bilinear alt on d succeeded
[2022-02-09 11:47:55.892] [console] [info] epsilon=-0.1565479193307659 is feasible
[2022-02-09 11:47:55.893] [console] [info] epsilon exceeds components of epsilon m

In [29]:
cspace_free_region_solution_bilinear = cspace_free_region.CspacePolytopeBilinearAlternation(q_star,
                                                                     filtered_collision_pairs,
                                                                     cspace_free_region_solution_round_robin.C,
                                                                    0.75*cspace_free_region_solution_round_robin.d,
                                                                     bilinear_alternation_options,
                                                                     solver_options, s)

[2022-02-08 19:34:36.721] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:34:36.723] [console] [info] Lagrangian step time 3.045 s
[2022-02-08 19:34:36.751] [console] [info] max(power(det(P), 1/4))=0.2071381570004865, solver_time 0.018611907958984375
[2022-02-08 19:34:36.753] [console] [info] cost improvement inf
[2022-02-08 19:34:38.615] [console] [info] Iter: 0, polytope step cost -0.008828768121858371, solver time 0.48531293869018555
[2022-02-08 19:34:41.859] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:34:41.860] [console] [info] Lagrangian step time 3.218 s
[2022-02-08 19:34:41.881] [console] [info] max(power(det(P), 1/4))=0.2105033555416147, solver_time 0.013540029525756836
[2022-02-08 19:34:41.882] [console] [info] cost improvement 0.0033651985411282115
[2022-02-08 19:34:43.780] [console] [info] Iter: 1, polytope step cost -0.008522369595631428, solver time 0.532606840133667
[2022-02-08 19:34:46.920] [console

[2022-02-08 19:36:00.640] [console] [info] Iter: 16, polytope step cost -0.00557567747292513, solver time 0.38878607749938965
[2022-02-08 19:36:04.137] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:36:04.141] [console] [info] Lagrangian step time 3.472 s
[2022-02-08 19:36:04.172] [console] [info] max(power(det(P), 1/4))=0.24350116763464993, solver_time 0.01593494415283203
[2022-02-08 19:36:04.174] [console] [info] cost improvement 0.001400599503426303
[2022-02-08 19:36:05.966] [console] [info] Iter: 17, polytope step cost -0.005466929346269455, solver time 0.3860509395599365
[2022-02-08 19:36:09.113] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:36:09.114] [console] [info] Lagrangian step time 3.119 s
[2022-02-08 19:36:09.142] [console] [info] max(power(det(P), 1/4))=0.24485582644352566, solver_time 0.00913095474243164
[2022-02-08 19:36:09.145] [console] [info] cost improvement 0.001354658808875725
[2022-02-08 19:3

In [31]:
if do_viz:
    visualizer.plot_regions([HPolyhedron(cspace_free_region_solution_bilinear.C,
                                         cspace_free_region_solution_bilinear.d)], 
                            ellipses=None, region_suffix='_bilinear')

In [15]:
regions_round_robin = []
ellipses_round_robin = []
for sol in cspace_free_region_solution_round_robin_vect:
    r = HPolyhedron(np.vstack([starting_poly.A(),sol.C]),
                np.hstack([starting_poly.b(), sol.d])
                )
    e = Hyperellipsoid(np.linalg.inv(sol.P), sol.q)

    regions_round_robin.append(r)
    ellipses_round_robin.append(e)
if do_viz:
    visualizer.plot_regions(regions_round_robin, 
                            ellipses=ellipses_round_robin, region_suffix='_round_robin')

In [22]:
if do_viz:
    visualizer.plot_regions([HPolyhedron(cspace_free_region_solution.C, 0.5*cspace_free_region_solution.d)], 
                            ellipses=None, region_suffix='_round_robin')

In [26]:
cspace_free_region_solution_vector_bisection = cspace_free_region.CspacePolytopeBisectionSearchVector(q_star,
                                                                     filtered_collision_pairs,
                                                                     C, d, 
                                                                     vector_bisection_search_options,
                                                                     solver_options, s)

[2022-02-07 15:27:37.125] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-07 15:27:37.194] [console] [info] max(power(det(P), 1/4))=-nan, solver_time 0.05913114547729492
[2022-02-07 15:27:40.385] [console] [info] bilinear alt on d succeeded
[2022-02-07 15:27:40.411] [console] [info] max(power(det(P), 1/4))=1.195363453968912e-06, solver_time 0.014796972274780273
[2022-02-07 15:27:45.796] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_3_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_oneDOF::iiwa_oneDOF_link_6_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)

[2022-02-07 15:27:45.797] [console] [info] total iter=1, feasible iter = 0
[2022-02-07 15:27:51.001] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_

In [29]:
if do_viz:
    visualizer.plot_regions([HPolyhedron(cspace_free_region_solution_vector_bisection.C, 
                                         cspace_free_region_solution_vector_bisection.d)], 
                            ellipses=None, region_suffix='_vector_bisection')

(True, <SolutionResult.kSolutionFound: 0>)


In [96]:
i

2